In [ ]:
class opencvsr:
    def __init__(self, model, scale):
        self.model = 
def upscale_normal_pic(src, dst, )

In [ ]:
import argparse
import time
import cv2
from cv2 import dnn_superres
import os
from utils import crop


def super_res_image(img_path, out_path, model_path):
    modelName = model_path.split("/")[-1].split("_")[0].lower()
    modelScale = model_path.split("_x")[-1]
    modelScale = int(modelScale[:modelScale.find(".")])
    sr = dnn_superres.DnnSuperResImpl_create()
    sr.readModel(model_path)
    sr.setModel(modelName, modelScale)
    _, filename = os.path.split(img_path)
    cols=600
    rows=600

    if os.path.splitext(filename)[-1] not in (".png", ".jpg", ".jpeg", ".bmp"):
        print("can't open file: {filename}")
    else:
        orig = cv2.imread(img_path, 1)
        if orig.shape[0]*orig.shape[1] <= cols*rows:
            upscaled = sr.upsample(orig)
            out_filename = filename.split(".")[0] + "_" + str(modelName) + "_x" + str(modelScale) + ".png"
            cv2.imwrite(out_path + "/" + str(out_filename), upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        else:
            # crop, sr then merge
            crop_path, num_cols, num_rows = crop.clip_one_picture(path=os.path.split(img_path)[0],filename=os.path.split(img_path)[-1],cols=cols,rows=rows)
            for i, filename in enumerate(os.listdir(crop_path)):
                img_path = crop_path + filename
                croped = cv2.imread(img_path, 1)
                upscaled = sr.upsample(croped)
                cv2.imwrite(img_path, upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
                
            merge_upscaled = crop.merge_picture(crop_path,sum_cols=modelScale*orig.shape[0],sum_rows=modelScale*orig.shape[1],cols=modelScale*cols,rows=modelScale*rows)
            out_filename = filename.split(".")[0] + "_" + str(modelName) + "_x" + str(modelScale) + ".png"
            cv2.imwrite(out_path + "/" + str(out_filename), merge_upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

def super_res(file_path, out_path, model_path):

    # modelName = model_path.split("/")[-1].split("_")[0].lower()
    # modelScale = model_path.split("_x")[-1]
    # modelScale = int(modelScale[:modelScale.find(".")])
    # sr = dnn_superres.DnnSuperResImpl_create()
    # sr.readModel(model_path)
    # sr.setModel(modelName, modelScale)

    # check output and input(file or dir) path
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    if os.path.isfile(file_path):
        super_res_image(img_path=file_path, out_path=out_path, model_path=model_path)
    elif os.path.isdir(file_path):
        for filename in os.listdir(file_path):
            img_path = file_path + "/" + filename
            super_res_image(img_path=img_path, out_path=out_path, model_path=model_path)
    else:
        print(f"cannot read path: {file_path}, please try again.")

    return


def main():
    model_path = "./models/FSRCNN_x3.pb"
    file_path = "./examples/" 
    img_path = file_path + "pexels-any-lane-5727731.png"
    out_path = file_path + "result"
    super_res(file_path, out_path, model_path)
    print("success!!!")


if __name__ == "__main__":
    main()